In [16]:
import pymysql
from tqdm import tqdm
import pyodbc

DB_CONNECTION = 'mysql'
DB_HOST = 'localhost'
DB_PORT = 3306
DB_DATABASE = '***'
DB_USERNAME = '***'
DB_PASSWORD = '***'

field_type = {
 1: 'INTEGER',
 2: 'INTEGER',
 3: 'INTEGER',
 5: 'DOUBLE',
 7: 'TIMESTAMP',
 12: 'TIMESTAMP',
 16: 'BIT',
 252: 'LONGTEXT',
 253: 'VARCHAR(255)',
 }

conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\\Users\\CBDB\Desktop\\mysql2access\\cbdb.mdb;'
    )
cnxn = pyodbc.connect(conn_str)
crsr = cnxn.cursor()

In [17]:
def get_table_name():
    db = pymysql.connect(host=DB_HOST, user=DB_USERNAME, password=DB_PASSWORD, database=DB_DATABASE,charset="utf8mb4",cursorclass=pymysql.cursors.DictCursor)
    cursor = db.cursor()
    cursor.execute("show tables")
    data = cursor.fetchall()
#     print(len(data))
#     print(data)
    db.close()
    return data

In [18]:
def get_columns(table_name):
    res = ""
    db = pymysql.connect(host=DB_HOST, user=DB_USERNAME, password=DB_PASSWORD, database=DB_DATABASE,charset="utf8mb4",cursorclass=pymysql.cursors.DictCursor)
    cursor = db.cursor()
    sqlSentence = "select * from `%s` limit 1"%table_name
    cursor.execute(sqlSentence)
    dataSchema = cursor.description
    for row in dataSchema:
        column_name = row[0]
        type_name = field_type[row[1]]
        res += "`%s` %s," % (column_name, type_name)
    res = res[:-1]
    sql = "CREATE TABLE `%s` ( %s )" % (table_name, res)
    db.close()
    return sql

def creat_table(table_name, sql, table_list):
    if table_name in table_list:
        print("DROP TABLE `%s`" % (table_name))
        crsr.execute("DROP TABLE `%s`" % (table_name))
        cnxn.commit()
        print('drop %s successed' % (table_name))
    crsr.execute(sql)
    cnxn.commit()
    print('create %s successed' % (table_name))

def get_values(table_name):
    db = pymysql.connect(host=DB_HOST, user=DB_USERNAME, password=DB_PASSWORD, database=DB_DATABASE,charset="utf8mb4",cursorclass=pymysql.cursors.DictCursor)
    cursor = db.cursor()
    cursor.execute('SELECT * FROM `%s`' % (table_name))
    data = cursor.fetchall()
    db.close()
    return data

def insert_values(table_name, rows):
    count = 0
    isNum = False
    for i in tqdm(range(len(rows))):
        row = []
        try:
            for _,k in rows[i].items():
                row.append(k)
            for j in range(len(row)):
                if row[j] == b'\x00':
                    row[j] = 0
                elif row[j] == b'\x01':
                    row[j] = 1
            query = "INSERT INTO `"+table_name+"` values ("+','.join(["?"]*len(row))+");"
            crsr.execute(query,tuple(row))
            count += 1
        except:
            print("stop at:")
            print(i)
            print(rows[i])
            print(row)
            print("INSERT INTO `"+table_name+"` values ("+','.join(["?"]*len(row))+");")
            raise
    cnxn.commit()


In [ ]:
def main():
    table_list = []
    for table_info in crsr.tables(tableType='TABLE'):
        table_list.append(table_info.table_name)
    print(table_list)
    skip_mysql_table_list = ["CBDB_NAME_LIST", "CBDB__NAME_FTS", "CBDB__TRAD_SIMP_MAP", "users", "operations", "password_resets", "pinyin", "migrations", "oauth_access_tokens", "oauth_auth_codes", "oauth_clients", "oauth_personal_access_clients", "oauth_refresh_tokens"]
    for table_name in get_table_name():
        if table_name_in_dic in skip_mysql_table_list:
            print(f"Skipped table: {table_name_in_dic}")
            continue
        for _,table_name_in_dic in table_name.items():
            # if table_name_in_dic != "SOCIAL_INSTITUTION_ADDR": continue
            table_name = table_name_in_dic.upper()
            try:
                creat_table_sql = get_columns(table_name)
            except:
                print(f"Skipped table: {table_name_in_dic}")
                continue
            creat_table(table_name, creat_table_sql, table_list)
            rows = get_values(table_name)
            insert_values(table_name, rows)
    cnxn.close()

In [20]:
main()

[]
create SOCIAL_INSTITUTION_ADDR successed


100%|██████████| 3783/3783 [00:04<00:00, 865.55it/s]
